***
## RT Beacons
***

In [1]:
using ODBC
using DataFrames
using DSWB
using Formatting

dsn = "dswb-natgeo" # Redshift endpoint
table = "beacons_4744" # beacon table name
tableRt = "beacons_4744_rt"

# Connect to Beacon Data
setRedshiftEndpoint(dsn)
setTable(tableRt, tableType = "RESOURCE_TABLE")
#setTable(table)

Redshift endpoint set: 2017-06-09T20:32:44


"beacons_4744_rt"

***
## 1.2 Beacons Table & Schema
***

In [2]:
# Packages
include("/data/notebook_home/Production/Lib/Include-Package-v1.0.jl")
include("/data/notebook_home/Production/Bodies/SQL-Data-Mining-Rt-Spike-Body-v1.0.jl")

In [3]:
# Create temp tables for Normal timeframe and Spike timeframe

urlNormal = UrlParamsInit()
urlSpike = UrlParamsInit()
urlCache = UrlParamsInit()
urlSummary = UrlParamsInit()

urlNormal.resourceTable = tableRt
urlNormal.rtView = "$(table)_rtnormal_view_prod"

urlSpike.resourceTable = tableRt
urlSpike.rtView = "$(table)_rtspike_view_prod"

timeNormal = anyTimeVar(2017,6,7,11,49,2017,6,7,11,59)
timeSpike  = anyTimeVar(2017,6,8,11,49,2017,6,8,11,59)

query("""create or replace view $(urlNormal.rtView) as (select * from $(urlNormal.resourceTable) where "timestamp" between $(timeNormal.startTimeMsUTC) and $(timeNormal.endTimeMsUTC))""")
query("""create or replace view $(urlSpike.rtView)  as (select * from $(urlSpike.resourceTable)  where "timestamp" between  $(timeSpike.startTimeMsUTC) and  $(timeSpike.endTimeMsUTC))""")


June 7, 2017 11:49:00 to June 7, 2017 11:59:00 Local Time
June 7, 2017 15:49:00 to June 7, 2017 15:59:00 UTC Time
June 8, 2017 11:49:00 to June 8, 2017 11:59:00 Local Time
June 8, 2017 15:49:00 to June 8, 2017 15:59:00 UTC Time


""


In [4]:
query("""SELECT count(*) FROM $(urlNormal.rtView)""")

,count
1,847812


In [5]:
query("""SELECT count(*) FROM $(urlSpike.rtView)""")

,count
1,761511


***
## Request Counters By URL Group
***

In [6]:
requestCountByGroupSDMRS(timeNormal,urlNormal,"Normal")
requestCountByGroupSDMRS(timeSpike,urlSpike,"Spike")

Normal: Request Counts By URL Group June 7, 2017 11:49:00 to June 7, 2017 11:59:00 Local Time

reqcnt,urlgroup
"66,097",http://kids.nationalgeographic.com
"60,040",http://www.nationalgeographic.com
"51,670",http://yourshot.nationalgeographic.com
"46,596",http://www.google-analytics.com
"33,112",https://securepubads.g.doubleclick.net
"30,233",https://www.facebook.com
"27,295",http://assets.nationalgeographic.com
"27,259",https://tpc.googlesyndication.com
"24,846",http://channel.nationalgeographic.com
"23,708",http://news.nationalgeographic.com


Spike: Request Counts By URL Group June 8, 2017 11:49:00 to June 8, 2017 11:59:00 Local Time

reqcnt,urlgroup
"58,353",http://kids.nationalgeographic.com
"51,512",http://www.nationalgeographic.com
"42,558",http://www.google-analytics.com
"42,251",http://yourshot.nationalgeographic.com
"33,251",https://securepubads.g.doubleclick.net
"28,707",https://tpc.googlesyndication.com
"26,799",https://www.facebook.com
"26,285",http://news.nationalgeographic.com
"21,764",http://assets.nationalgeographic.com
"18,715",http://fonts.ngeo.com


***
## Blocking Requests By URL Groups Across All Sessions
***

In [7]:
blockingRequestCountByGroupSDMRS(timeNormal,urlNormal,"Normal")
blockingRequestCountByGroupSDMRS(timeSpike,urlSpike,"Spike")
;

Normal: Blocking Requests By URL Groups Across All Sessions June 7, 2017 11:49:00 to June 7, 2017 11:59:00 Local Time

reqcnt,totalblk,avgblk,urlgroup
"28,480","38,601,803","1,355",http://yourshot.nationalgeographic.com
"16,549","29,472,136","1,780",http://channel.nationalgeographic.com
"44,404","12,906,404",290,http://www.nationalgeographic.com
"47,291","12,666,678",267,http://kids.nationalgeographic.com
"20,209","3,649,382",180,http://news.nationalgeographic.com
"30,385","3,168,028",104,https://securepubads.g.doubleclick.net
"12,590","2,739,873",217,http://www.google-analytics.com
"3,670","2,376,145",647,http://voices.nationalgeographic.com
"7,752","1,966,752",253,http://s7.addthis.com
"1,161","1,869,640","1,610",http://images.nationalgeographic.com


Spike: Blocking Requests By URL Groups Across All Sessions June 8, 2017 11:49:00 to June 8, 2017 11:59:00 Local Time

reqcnt,totalblk,avgblk,urlgroup
"25,568","49,618,141","1,940",http://yourshot.nationalgeographic.com
"12,485","20,562,042","1,646",http://channel.nationalgeographic.com
"42,201","14,307,493",339,http://kids.nationalgeographic.com
"37,727","8,233,700",218,http://www.nationalgeographic.com
"1,317","4,732,990","3,593",http://images.nationalgeographic.com
"22,971","3,854,633",167,http://news.nationalgeographic.com
"3,777","3,550,618",940,http://voices.nationalgeographic.com
"30,866","3,355,308",108,https://securepubads.g.doubleclick.net
"11,378","2,339,105",205,http://www.google-analytics.com
"14,818","1,964,022",132,https://tpc.googlesyndication.com


***
## Non Cache Requests Total By URL Groups Across All Sessions
***

In [8]:
nonCacheRequestCountByGroupSDMRS(timeNormal,urlNormal,"Normal")
nonCacheRequestCountByGroupSDMRS(timeSpike,urlSpike,"Spike")
;

Normal: Non Cache Requests Total By URL Groups Across All Sessions June 7, 2017 11:49:00 to June 7, 2017 11:59:00 Local Time

count,urlgroup
"49,134",http://www.nationalgeographic.com
"42,432",http://www.google-analytics.com
"33,197",http://kids.nationalgeographic.com
"29,834",https://www.facebook.com
"27,059",http://yourshot.nationalgeographic.com
"25,767",https://securepubads.g.doubleclick.net
"24,959",http://assets.nationalgeographic.com
"19,351",http://news.nationalgeographic.com
"18,439",http://cdn.video.nationalgeographic.com
"16,059",http://channel.nationalgeographic.com


Spike: Non Cache Requests Total By URL Groups Across All Sessions June 8, 2017 11:49:00 to June 8, 2017 11:59:00 Local Time

count,urlgroup
"42,704",http://www.nationalgeographic.com
"39,038",http://www.google-analytics.com
"32,824",http://kids.nationalgeographic.com
"26,448",https://www.facebook.com
"26,090",https://securepubads.g.doubleclick.net
"24,344",http://yourshot.nationalgeographic.com
"21,540",http://news.nationalgeographic.com
"20,175",http://assets.nationalgeographic.com
"14,959",https://tpc.googlesyndication.com
"14,815",http://fonts.ngeo.com


***
##Create the Cache Ratio
***

In [9]:
cacheHitRatioSDMRS(timeNormal,urlNormal,"Normal")
cacheHitRatioSDMRS(timeSpike,urlSpike,"Spike")
;

Normal: Cache Hit Ratio By URL Groups Across All Sessions June 7, 2017 11:49:00 to June 7, 2017 11:59:00 Local Time

Url Group,Not Cached Cnt,Cached Cnt,Cached Ratio
http://www.nationalgeographic.com,"49,134","10,832",18.063569
http://www.google-analytics.com,"42,432","4,161",8.930526
http://kids.nationalgeographic.com,"33,197","32,877",49.757847
https://www.facebook.com,"29,834",393,1.300162
http://yourshot.nationalgeographic.com,"27,059","24,594",47.613885
https://securepubads.g.doubleclick.net,"25,767","7,332",22.151727
http://assets.nationalgeographic.com,"24,959","2,336",8.558344
http://news.nationalgeographic.com,"19,351","4,344",18.332982
http://cdn.video.nationalgeographic.com,"18,439","1,638",8.158589
http://channel.nationalgeographic.com,"16,059","8,770",35.3216


Spike: Cache Hit Ratio By URL Groups Across All Sessions June 8, 2017 11:49:00 to June 8, 2017 11:59:00 Local Time

Url Group,Not Cached Cnt,Cached Cnt,Cached Ratio
http://www.nationalgeographic.com,"42,704","8,764",17.028056
http://www.google-analytics.com,"39,038","3,512",8.253819
http://kids.nationalgeographic.com,"32,824","25,512",43.732858
https://www.facebook.com,"26,448",342,1.276596
https://securepubads.g.doubleclick.net,"26,090","7,144",21.496058
http://yourshot.nationalgeographic.com,"24,344","17,867",42.327829
http://news.nationalgeographic.com,"21,540","4,735",18.020932
http://assets.nationalgeographic.com,"20,175","1,589",7.301048
https://tpc.googlesyndication.com,"14,959","13,735",47.86715
http://fonts.ngeo.com,"14,815","3,900",20.838899


***
## All Done
***

In [10]:
q = query(""" drop view if exists $(urlNormal.rtView);""")
q = query(""" drop view if exists $(urlSpike.rtView);""")
;